In [4]:
from IPython.core.display import display, HTML
display(HTML("""
    <script>
        var kernel = IPython.notebook.kernel;
        kernel.execute('from IPython.core.display import clear_output');
    </script>
"""))

import IPython
IPython.get_ipython().config.TerminalInteractiveShell.rate_limit_window = 0.0

/tmp/ipykernel_2280002/4248719676.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
import pandas as pd
df = pd.read_csv("pred_data_preprocessed.csv")
# Count rows where dicta_answer is 1

In [9]:
df = df.drop("Unnamed: 0",axis=1)
df = df.drop("position",axis=1)

In [10]:
df

,committee_name,session_id,chairperson,speaker_name,conversation,contain_offensive_words,dicta_answer
0,אל על,64670,אברהם הירשזון,אברהם הירשזון,. אני כן אסתום לך את הפה.. .,0,0.0
1,אל על,64670,אברהם הירשזון,יצחק כהן,". לא, זאת הצעה לסדר. כל הדיון הזה מיותר. אני מ...",0,0.0
2,אל על,64670,אברהם הירשזון,רוחמה אברהם,". אם חברה הולכת להפרטה, אסור לה לפרסם תשקיף..",0,0.0
3,אל על,64670,אברהם הירשזון,אברהם הירשזון,. אני קורא אותך לסדר..,1,1.0
4,אל על,64670,אברהם הירשזון,יצחק כהן,". לא, זאת הצעה לסדר. כל הדיון הזה מיותר. אני מ...",0,0.0
...,...,...,...,...,...,...,...
127835,ועדת החוקה חוק ומשפט,2225681,שמחה רוטמן,קריאות,\n- - -\n,0,0.0
127836,ועדת החוקה חוק ומשפט,2225681,שמחה רוטמן,גלעד קריב,\nמין שאינו במינו.\n,0,0.0
127837,ועדת החוקה חוק ומשפט,2225681,שמחה רוטמן,שמחה רוטמן,\nההערה הזאת צרמה לי מאוד באוזן על המישור המקצ...,0,0.0
127838,ועדת החוקה חוק ומשפט,2225681,שמחה רוטמן,אלעזר שטרן,"\nאדוני, אני אשמח להגיב לדבר הזה. קודם כול, אנ...",0,0.0


## Model

## Baseline

In [15]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
import os
import json
import logging
from datetime import datetime
import sys
from torch.amp import autocast, GradScaler
import gc
import torch.cuda as cuda


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc


class DictaDataset(Dataset):
    def __init__(self, conversations, labels, tokenizer, max_length=256):
        self.conversations = conversations
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.conversations)

    def __getitem__(self, idx):
        conversation = str(self.conversations[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            conversation,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

def calculate_class_weights(y_train):
    total_samples = len(y_train)
    class_counts = np.bincount(y_train)
    class_weights = torch.FloatTensor(total_samples / (len(class_counts) * class_counts))
    return class_weights

def setup_logging(base_dir='results_baseline'):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_dir = os.path.join(base_dir, f'run_{timestamp}')
    os.makedirs(results_dir, exist_ok=True)
    
    log_file = os.path.join(results_dir, 'training.log')
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler(sys.stdout)
        ]
    )
    
    plots_dir = os.path.join(results_dir, 'plots')
    models_dir = os.path.join(results_dir, 'models')
    metrics_dir = os.path.join(results_dir, 'metrics')
    
    os.makedirs(plots_dir, exist_ok=True)
    os.makedirs(models_dir, exist_ok=True)
    os.makedirs(metrics_dir, exist_ok=True)
    
    return results_dir, plots_dir, models_dir, metrics_dir

def save_metrics(metrics, file_path):
    metrics_dict = {
        'accuracy': float(metrics['accuracy']),
        'precision': float(metrics['precision']),
        'recall': float(metrics['recall']),
        'f1': float(metrics['f1']),
        'roc_auc': float(metrics['roc_auc']),
        'confusion_matrix': metrics['confusion_matrix'].tolist(),
        'classification_report': metrics['classification_report'],
        'roc_curve': metrics['roc_curve']
    }
    
    with open(file_path, 'w') as f:
        json.dump(metrics_dict, f, indent=4)
        
def evaluate_during_training(model, dataloader, device, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            
    return total_loss / len(dataloader)

def train_model(train_dataloader, val_dataloader, model, optimizer, device, num_epochs=3, results_dir=None, gradient_accumulation_steps=4, patience=3):
    model.train()
    training_stats = []
    scaler = GradScaler('cuda')
    early_stopping = EarlyStopping(patience=patience)
    
    # Create scheduler with warmup
    num_training_steps = len(train_dataloader) * num_epochs
    num_warmup_steps = num_training_steps // 5  # 20% of training steps for warmup
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=num_warmup_steps, 
        num_training_steps=num_training_steps
    )
    
    logging.info(f"Total training steps: {num_training_steps}")
    logging.info(f"Warmup steps: {num_warmup_steps}")
    
    for epoch in range(num_epochs):
        total_loss = 0
        epoch_stats = {'epoch': epoch + 1, 'batch_losses': []}
        
        pbar = tqdm(train_dataloader, 
                   desc=f'Epoch {epoch + 1}/{num_epochs}',
                   total=len(train_dataloader),
                   dynamic_ncols=True,
                   position=0,
                   leave=True,
                   mininterval=1.0)
        
        optimizer.zero_grad()
        
        for batch_idx, batch in enumerate(pbar):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Clear cache if needed
            if batch_idx % 50 == 0:
                gc.collect()
                torch.cuda.empty_cache()

            with autocast('cuda'):
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )
                loss = outputs.loss / gradient_accumulation_steps

            scaler.scale(loss).backward()
            
            if (batch_idx + 1) % gradient_accumulation_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()
            
            total_loss += loss.item() * gradient_accumulation_steps
            epoch_stats['batch_losses'].append(loss.item() * gradient_accumulation_steps)
            
            current_lr = scheduler.get_last_lr()[0]
            pbar.set_postfix_str(f'loss={loss.item() * gradient_accumulation_steps:.4f}, lr={current_lr:.2e}', refresh=True)
            
        avg_loss = total_loss / len(train_dataloader)
        epoch_stats['average_loss'] = avg_loss
        training_stats.append(epoch_stats)
        
        # Validation phase
        val_loss = evaluate_during_training(model, val_dataloader, device, outputs.loss)
        epoch_stats['validation_loss'] = val_loss
        
        pbar.close()
        logging.info(f'Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_loss:.4f} - Val Loss: {val_loss:.4f}')
        
        if results_dir:
            stats_file = os.path.join(results_dir, 'metrics', f'epoch_{epoch + 1}_stats.json')
            with open(stats_file, 'w') as f:
                json.dump(epoch_stats, f, indent=4)
        
        # Save checkpoint
        if epoch_stats.get('validation_loss', float('inf')) == early_stopping.best_loss:
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': avg_loss,
            }
            torch.save(checkpoint, os.path.join(results_dir, 'models', f'checkpoint_best.pt'))
        
        # Early stopping check
        early_stopping(val_loss)
        if early_stopping.early_stop:
            logging.info(f'Early stopping triggered after epoch {epoch + 1}')
            break
                
        gc.collect()
        torch.cuda.empty_cache()
    
    return training_stats
    
def evaluate_model(model, test_dataloader, device, plots_dir=None, phase='test'):
    """
    Evaluate the model's performance on a given dataset.
    
    Args:
        model: The trained model to evaluate
        test_dataloader: DataLoader containing the evaluation data
        device: Device to run the evaluation on (cuda/cpu)
        plots_dir: Directory to save plots (optional)
        phase: String indicating the evaluation phase ('test', 'validation', etc.)
    
    Returns:
        Dictionary containing evaluation metrics
    """
    model.eval()
    predictions = []
    actual_labels = []
    probabilities = []  # Added to store prediction probabilities
    total_batches = len(test_dataloader)
    
    # Create progress bar
    progress_bar = tqdm(
        enumerate(test_dataloader), 
        total=total_batches,
        desc=f'Evaluating {phase} set',
        dynamic_ncols=True,
        leave=True
    )
    
    # Evaluation loop
    with torch.no_grad():
        for batch_idx, batch in progress_bar:
            # Move data to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels']

            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            # Get predictions and probabilities
            probs = torch.softmax(outputs.logits, dim=1)
            _, preds = torch.max(outputs.logits, dim=1)
            
            # Store predictions, probabilities, and labels
            predictions.extend(preds.cpu().tolist())
            probabilities.extend(probs[:, 1].cpu().tolist())  # Store probability of positive class
            actual_labels.extend(labels.tolist())
            
            # Update progress bar with completion percentage
            progress_bar.set_postfix({
                'progress': f'{(batch_idx + 1)/total_batches:.1%}'
            })
            
            # Periodically clear GPU memory
            if (batch_idx + 1) % 50 == 0:
                torch.cuda.empty_cache()
    
    progress_bar.close()
    
    # Calculate metrics
    accuracy = accuracy_score(actual_labels, predictions)
    precision = precision_score(actual_labels, predictions, average='binary')
    recall = recall_score(actual_labels, predictions, average='binary')
    f1 = f1_score(actual_labels, predictions, average='binary')
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(actual_labels, probabilities)
    roc_auc = auc(fpr, tpr)
    
    # Create and save ROC curve plot
    if plots_dir:
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, color='darkorange', lw=2, 
                label=f'ROC curve (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve - {phase.capitalize()} Set')
        plt.legend(loc="lower right")
        plt.savefig(os.path.join(plots_dir, f'roc_curve_{phase}.png'), 
                   bbox_inches='tight', 
                   dpi=300)
        plt.close()

        # Create and plot confusion matrix
        cm = confusion_matrix(actual_labels, predictions)
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm, 
            annot=True, 
            fmt='d', 
            cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive']
        )
        plt.title(f'Confusion Matrix - {phase.capitalize()} Set')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.savefig(os.path.join(plots_dir, f'confusion_matrix_{phase}.png'), 
                   bbox_inches='tight', 
                   dpi=300)
        plt.close()
    
    # Log results
    logging.info(f'\n{phase.capitalize()} Set Metrics:')
    logging.info(f'Accuracy: {accuracy:.4f}')
    logging.info(f'Precision: {precision:.4f}')
    logging.info(f'Recall: {recall:.4f}')
    logging.info(f'F1 Score: {f1:.4f}')
    logging.info(f'ROC AUC: {roc_auc:.4f}')
    
    # Generate classification report
    class_report = classification_report(
        actual_labels, 
        predictions,
        target_names=['Negative', 'Positive'],
        digits=4
    )
    logging.info(f'\nClassification Report:\n{class_report}')
    
    # Prepare and return results dictionary
    results = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'roc_auc': roc_auc,
        'confusion_matrix': cm,
        'classification_report': class_report,
        'predictions': predictions,
        'actual_labels': actual_labels,
        'probabilities': probabilities,
        'roc_curve': {
            'fpr': fpr.tolist(),
            'tpr': tpr.tolist()
        }
    }
    
    return results
    
def predict_single_conversation(conversation, model, tokenizer, device):
    model.eval()
    encoding = tokenizer.encode_plus(
        conversation,
        add_special_tokens=True,
        max_length=256,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        _, preds = torch.max(outputs.logits, dim=1)
    
    return preds.item()

def main():
    # Setup logging and directories
    results_dir, plots_dir, models_dir, metrics_dir = setup_logging()
    logging.info("Starting training process...")
    
    # Load your data
    logging.info(f"Loaded dataset with {len(df)} samples")
    
    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-multilingual-cased',
        num_labels=2
    )
    
    # Log BERT initialization message
    logging.info("Note: Classifier layer is initialized randomly as expected for fine-tuning")
    
    # Prepare data
    X = df['conversation'].values
    y = df['dicta_answer'].values.astype(int)
    
    # Calculate class weights
    class_weights = calculate_class_weights(y)
    logging.info(f"Class weights: {class_weights.tolist()}")
    
    # First split: 80% train+val, 20% test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Second split: Split the 80% into 75% train, 25% val (60% and 20% of total)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, 
        test_size=0.25,
        random_state=42,
        stratify=y_train_val
    )
    
    # Log splits and class distribution
    logging.info(f"\nDataset splits:")
    logging.info(f"Training samples: {len(X_train)} ({len(X_train)/len(X)*100:.1f}%)")
    logging.info(f"Validation samples: {len(X_val)} ({len(X_val)/len(X)*100:.1f}%)")
    logging.info(f"Test samples: {len(X_test)} ({len(X_test)/len(X)*100:.1f}%)")
    
    logging.info("\nClass distribution:")
    logging.info(f"Training set: {np.bincount(y_train)}")
    logging.info(f"Validation set: {np.bincount(y_val)}")
    logging.info(f"Test set: {np.bincount(y_test)}")
    
    # Create datasets
    train_dataset = DictaDataset(X_train, y_train, tokenizer)
    val_dataset = DictaDataset(X_val, y_val, tokenizer)
    test_dataset = DictaDataset(X_test, y_test, tokenizer)
    
    # Adjusted batch size and workers for large dataset
    batch_size = 128  # Increased batch size
    gradient_accumulation_steps = 4  # Increased accumulation steps
    
    # Create dataloaders
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,  # Increased workers
        pin_memory=True,
        prefetch_factor=2  # Add prefetching
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size * 2,  # Larger batch size for validation
        num_workers=4,
        pin_memory=True,
        prefetch_factor=2
    )
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size * 2,
        num_workers=4,
        pin_memory=True,
        prefetch_factor=2
    )
    
    # Setup device and model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f"Using device: {device}")
    
    # Enable gradient checkpointing for memory efficiency
    model.gradient_checkpointing_enable()
    model.to(device)
    
    # Move class weights to device
    class_weights = class_weights.to(device)
    
    # Adjusted learning rate and optimizer parameters
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=5e-5,  # Increased learning rate
        weight_decay=0.01,  # Added weight decay
        betas=(0.9, 0.999),  # Adjusted momentum parameters
    )
    
    # Train the model with adjusted parameters
    logging.info("\nStarting training...")
    training_stats = train_model(
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        model=model,
        optimizer=optimizer,
        device=device,
        num_epochs=3,
        results_dir=results_dir,
        gradient_accumulation_steps=gradient_accumulation_steps,
        patience=3
    )
    
    # Save training curves
    plt.figure(figsize=(10, 6))
    plt.plot([stat['epoch'] for stat in training_stats], 
             [stat['average_loss'] for stat in training_stats],
             label='Training Loss')
    plt.plot([stat['epoch'] for stat in training_stats],
             [stat['validation_loss'] for stat in training_stats],
             label='Validation Loss')
    plt.title('Training and Validation Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(plots_dir, 'training_loss.png'))
    plt.close()
    
    # Load best model from checkpoint
    best_model_path = os.path.join(results_dir, 'models', 'checkpoint_best.pt')
    if os.path.exists(best_model_path):
        checkpoint = torch.load(best_model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        logging.info(f"Loaded best model from epoch {checkpoint['epoch']}")
    
    # Evaluate on validation set
    logging.info("\nEvaluating on validation set...")
    val_results = evaluate_model(model, val_dataloader, device, plots_dir, 'validation')
    save_metrics(val_results, os.path.join(metrics_dir, 'validation_metrics.json'))
    
    # Evaluate on test set
    logging.info("\nEvaluating on test set...")
    test_results = evaluate_model(model, test_dataloader, device, plots_dir, 'test')
    save_metrics(test_results, os.path.join(metrics_dir, 'test_metrics.json'))
    
    # Save the final model and tokenizer
    model_save_path = os.path.join(models_dir, 'final_model')
    tokenizer_save_path = os.path.join(models_dir, 'tokenizer')
    
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(tokenizer_save_path)
    
    # Save model configuration
    config = {
        'model_name': 'bert-base-multilingual-cased',
        'max_length': 256,
        'num_labels': 2,
        'batch_size': batch_size,
        'gradient_accumulation_steps': gradient_accumulation_steps,
        'learning_rate': 5e-5,
        'weight_decay': 0.01,
        'early_stopping_patience': 3,
        'num_epochs': 3,
        'train_size': len(X_train),
        'val_size': len(X_val),
        'test_size': len(X_test),
        'class_weights': class_weights.cpu().tolist()
    }
    
    with open(os.path.join(models_dir, 'model_config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    logging.info("Training and evaluation completed!")
    logging.info(f"Model saved to: {model_save_path}")
    logging.info(f"Tokenizer saved to: {tokenizer_save_path}")
    
    return model, tokenizer, {
        'train': training_stats, 
        'validation': val_results, 
        'test': test_results
    }

if __name__ == "__main__":
    main()

2025-02-05 13:57:59,879 - INFO - Starting training process...
2025-02-05 13:57:59,879 - INFO - Loaded dataset with 127840 samples


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2025-02-05 13:58:00,914 - INFO - Note: Classifier layer is initialized randomly as expected for fine-tuning
2025-02-05 13:58:00,915 - INFO - Class weights: [0.6072466969490051, 2.8310744762420654]
2025-02-05 13:58:00,954 - INFO - 
Dataset splits:
2025-02-05 13:58:00,955 - INFO - Training samples: 76704 (60.0%)
2025-02-05 13:58:00,955 - INFO - Validation samples: 25568 (20.0%)
2025-02-05 13:58:00,956 - INFO - Test samples: 25568 (20.0%)
2025-02-05 13:58:00,956 - INFO - 
Class distribution:
2025-02-05 13:58:00,957 - INFO - Training set: [63157 13547]
2025-02-05 13:58:00,957 - INFO - Validation set: [21053  4515]
2025-02-05 13:58:00,957 - INFO - Test set: [21052  4516]
2025-02-05 13:58:00,958 - INFO - Using device: cuda
2025-02-05 13:58:01,062 - INFO - 
Starting training...
2025-02-05 13:58:01,063 - INFO - Total training steps: 1800
2025-02-05 13:58:01,064 - INFO - Warmup steps: 360


Epoch 1/3: 100%|██████████| 600/600 [02:54<00:00,  3.44it/s, loss=0.1811, lr=2.08e-05]


2025-02-05 14:02:01,132 - INFO - Epoch 1/3 - Train Loss: 0.4344 - Val Loss: 0.3260


...

2025-02-05 14:08:29,756 - INFO - Epoch 3/3 - Train Loss: 0.2140 - Val Loss: 0.2509
2025-02-05 14:08:30,312 - INFO - 
Evaluating on validation set...


Evaluating validation set: 100%|██████████| 100/100 [01:05<00:00,  1.52it/s, progress=100.0%]


2025-02-05 14:09:36,761 - INFO - 
Validation Set Metrics:
2025-02-05 14:09:36,763 - INFO - Accuracy: 0.9058
2025-02-05 14:09:36,764 - INFO - Precision: 0.7416
2025-02-05 14:09:36,764 - INFO - Recall: 0.7163
2025-02-05 14:09:36,765 - INFO - F1 Score: 0.7287
2025-02-05 14:09:36,766 - INFO - ROC AUC: 0.9175
2025-02-05 14:09:36,796 - INFO - 
Classification Report:
              precision    recall  f1-score   support

    Negative     0.9396    0.9465    0.9430     21053
    Positive     0.7416    0.7163    0.7287      4515

    accuracy                         0.9058     25568
   macro avg     0.8406    0.8314    0.8359     25568
weighted avg     0.9046    0.9058    0.9052     25568

2025-02-05 14:09:36,813 - INFO - 
Evaluating on test set...


Evaluating test set: 100%|██████████| 100/100 [01:05<00:00,  1.53it/s, progress=100.0%]


2025-02-05 14:10:43,116 - INFO - 
Test Set Metrics:
2025-02-05 14:10:43,117 - INFO - Accuracy: 0.9084
2025-02-05 14:10:43,117 - INFO - Precision: 0.7477
2025-02-05 14:10:43,118 - INFO - Recall: 0.7270
2025-02-05 14:10:43,118 - INFO - F1 Score: 0.7372
2025-02-05 14:10:43,118 - INFO - ROC AUC: 0.9226
2025-02-05 14:10:43,138 - INFO - 
Classification Report:
              precision    recall  f1-score   support

    Negative     0.9418    0.9474    0.9446     21052
    Positive     0.7477    0.7270    0.7372      4516

    accuracy                         0.9084     25568
   macro avg     0.8447    0.8372    0.8409     25568
weighted avg     0.9075    0.9084    0.9079     25568

2025-02-05 14:10:46,168 - INFO - Training and evaluation completed!
2025-02-05 14:10:46,168 - INFO - Model saved to: results_baseline/run_20250205_135759/models/final_model
2025-02-05 14:10:46,169 - INFO - Tokenizer saved to: results_baseline/run_20250205_135759/models/tokenizer
